# Заполнятор досрочки

## Импорты

In [ ]:
import time, os
start = time.time()

import pandas as pd
import docxtpl
import pymorphy2

from pytrovich.enums import NamePart, Gender, Case
from pytrovich.maker import PetrovichDeclinationMaker
from pytrovich.detector import PetrovichGenderDetector

morph = pymorphy2.MorphAnalyzer()
maker = PetrovichDeclinationMaker()
detector = PetrovichGenderDetector()

## Google-таблица

In [ ]:
data = pd.read_csv('<place for table>')

In [ ]:
#print(data.shape)
data

## Функции

In [ ]:
def dative(row):
    dative_parser = morph.parse(row)[0]
    datv = dative_parser.inflect({'datv'})
    return datv.word

def genitive(row):
    words = row.split(' ')
    #print(words)
    final_list = []
    for word in words:
        gent_parser = morph.parse(word)[0]
        #print(gent_parser)
        gent = gent_parser.inflect({'gent'})
        #print(gent)
        final_list.append(gent)
    return final_list

def letter(row):
    return row[0] + '.'

def gender(df):
    genders = []
    for i in range(df.shape[0]):
        try:
            value = detector.detect(firstname=df['supervisor_name'][i], middlename=df['supervisor_patronymic'][i], lastname=df['supervisor_surname'][i])
            genders.append(value)
        except:
            value = 'NotPredicted'
            genders.append(value)
    return genders

def appeal(df):
    if df['supervisor_gender'] == Gender.FEMALE:
        return 'Уважаемая'
    else:
        return 'Уважаемый'

def dative_surname(df):
    if df['supervisor_gender'] == Gender.FEMALE:
        return maker.make(NamePart.LASTNAME, Gender.FEMALE, Case.DATIVE, df['supervisor_surname'])
    else: 
        return maker.make(NamePart.LASTNAME, Gender.MALE, Case.DATIVE, df['supervisor_surname'])

## Преобразования

In [ ]:
transformed_data = pd.DataFrame(columns=['time', 'position', 'organization', 'supervisor_n1', 'supervisor_p1', 'supervisor_surname', 'dear', 'supervisor_name', 'supervisor_patronymic', 'student_fullname', 'student_group', 'supervisor_gender'])

transformed_data['time'] = data.iloc[:,0]
transformed_data['supervisor_n1'] = data.iloc[:,9].apply(letter).str.strip(' ')
transformed_data['supervisor_p1'] = data.iloc[:,10].apply(letter).str.strip(' ')
transformed_data['supervisor_surname'] = data.iloc[:,8].str.strip(' ')
transformed_data['supervisor_name'] = data.iloc[:,9].str.strip(' ')
transformed_data['supervisor_patronymic'] = data.iloc[:,10].str.strip(' ')
transformed_data['student_fullname'] = data.iloc[:,2].str.strip(' ') + ' ' + data.iloc[:,3].str.strip(' ') + ' ' + data.iloc[:,4].str.strip(' ')
transformed_data['student_group'] = data.iloc[:,5]
transformed_data['supervisor_gender'] = gender(transformed_data)
transformed_data['dear'] = transformed_data.apply(appeal, axis=1)

transformed_data['position'] = data.iloc[:,6].apply(dative).str.capitalize()
transformed_data['organization'] = "«" + data.iloc[:,7].str.rstrip(' ') + '»'#.apply(genitive)
transformed_data['supervisor_surname'] = transformed_data.apply(dative_surname, axis=1)

display(transformed_data)

In [ ]:
transformed_data.to_csv(os.path.join('sources', 'test_csv.csv'))

In [ ]:
values_dictionary = transformed_data.reset_index().to_dict('records')
#values_dictionary

In [ ]:
test_dict = { 
    'position' : 'Ректор', 
    'organization' : 'РТУ МИРЭА', 
    'supervisor_n1': 'С.', 
    'supervisor_p1': 'А.', 
    'supervisor_surname': 'Кудж',
    'supervisor_name' : 'Станислав', 
    'supervisor_patronymic' : 'Алексеевич', 
    'student_fullname': 'Ермаков Сергей Романович', 
    'student_group': 'ИАМО-01-14',
    }

## Сохранение

In [ ]:
def save_docs(values_dictionary):
    saved_docs = []
    for context in values_dictionary:
        doc = docxtpl.DocxTemplate(os.path.join('sources', 'prescheduled.docx'))
        doc.render(context)
        saved_doc = os.path.join('output', 'prescheduled', 'ДС_' + context['student_fullname'] + '.docx')
        doc.save(saved_doc)
        saved_docs.append(saved_doc)
    print(saved_docs)
    return saved_docs

In [ ]:
save_docs(values_dictionary)

In [ ]:
end = time.time()
print('Время работы скрипта: ', round(end - start, 2), 'секунд')

## Источники

https://habr.com/ru/post/456534/

https://blog.formpl.us/how-to-generate-word-documents-from-templates-using-python-cb039ea2c890

https://habr.com/ru/post/483302/

http://helpexe.ru/programmirovanie/kak-chitat-i-pisat-v-google-sheets-s-python

https://fooobar.com/questions/130030/download-a-spreadsheet-from-google-docs-using-python

https://ru.stackoverflow.com/questions/934988/%D0%9A%D0%B0%D0%BA-%D1%8F-%D0%BC%D0%BE%D0%B3%D1%83-%D0%B4%D0%BE%D0%B1%D1%8B%D1%82%D1%8C-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5-%D1%81-google-docs-%D1%81-%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E-python

https://issue.life/questions/44992650

https://ru.stackoverflow.com/questions/1157933/%D0%A1%D0%BA%D0%BB%D0%BE%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5-%D1%81%D0%BB%D0%BE%D0%B2-%D0%BF%D0%BE-%D0%BF%D0%B0%D0%B4%D0%B5%D0%B6%D0%B0%D0%BC-%D0%B2-python

https://buildmedia.readthedocs.org/media/pdf/docxtpl/latest/docxtpl.pdf